# Lab 1: Building a ReAct Agent from Scratch

## The ReAct Pattern

In this section, we'll construct an AI agent using the ReAct (Reasoning and Acting) pattern. If you're unfamiliar with this concept, don't worry—we'll break it down step by step.

The ReAct pattern is a framework for structuring an AI's problem-solving process, mirroring human cognitive patterns:

1. **Reason** about the current situation
2. **Decide** on an action to take
3. **Observe** the results of that action
4. **Repeat** until the task is complete

To illustrate this concept, consider how an experienced software engineer might approach debugging a complex system:

1. **Reason**: Analyze the error logs and system state (e.g., "The database connection is timing out")
2. **Act**: Implement a diagnostic action (e.g., "Run a database connection test")
3. **Observe**: Examine the results of the diagnostic (e.g., "The test shows high latency")
4. **Repeat**: Continue this process, perhaps checking network configurations next, until the issue is resolved

Our AI agent will employ a similar methodology to tackle problems. As we develop this agent, pay attention to the division of labor between the AI model (the "brain" that reasons and decides) and our Python code (the "body" that interacts with the environment and manages the process flow).

This notebook is based on the following [notebook from Simon Willison](https://til.simonwillison.net/llms/python-react-pattern).


## Setting Up the Environment

Let's begin by importing the necessary libraries and configuring our environment.

### Initializing the Bedrock Client

To communicate with the Claude model via Amazon Bedrock, we need to establish a client connection. Think of this client as an API gateway that enables our code to send requests to the AI model and receive responses.

We'll use the `boto3` library, which is the Amazon Web Services (AWS) SDK for Python. For those unfamiliar with AWS, `boto3` can be thought of as a comprehensive toolkit that facilitates Python's interaction with various AWS services, including Bedrock.

For comprehensive instructions on configuring `boto3` with AWS credentials, refer to the [AWS documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html).

In a production setting, you would implement secure AWS credential management. For the purposes of this section, we'll assume the credentials are pre-configured in your environment.


In [ ]:
from dotenv import load_dotenv
import os
import sys
import boto3
import re
from botocore.config import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client in your aws region.
# If you do not have the AWS CLI profile setup, you can authenticate with aws access key, secret and session token.
# For more details check https://docs.aws.amazon.com/cli/v1/userguide/cli-authentication-short-term.html
bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    config=bedrock_config,
)

First we are going to define a few inference parameters and test our connection via `boto3` to Amazon Bedrock.


In [ ]:
# Set inference parameters
temperature = 0.0
top_k = 200
inference_config = {"temperature": temperature}

additional_model_fields = {"top_k": top_k}
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
system_prompts = [{"text": "You are a helpful agent."}]
message_1 = {"role": "user", "content": [{"text": "Hello world"}]}

# Instantiate messages list
messages = []
messages.append(message_1)

# Send the message.
response = bedrock_rt.converse(
    modelId=model_id,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields,
)

pp.pprint(response)
print("\n\n")
pp.pprint(response["output"]["message"]["content"][0]["text"])

## Designing the Agent Class

With our Bedrock client set up, we'll now create our Agent class. This class will serve as the core of our AI agent, encapsulating the logic for interacting with the Claude model and maintaining the conversation state.

The ReAct pattern, which our agent will implement, consists of three primary steps:

1. **Reasoning (Thought)**: The agent assesses the current situation and formulates a plan. For instance, "To calculate the total weight of two dog breeds, I need to look up their individual weights and then sum them."

2. **Acting (Action)**: Based on its reasoning, the agent selects an appropriate action. For example, "I will query the average weight of a Border Collie."

3. **Observing (Observation)**: The agent processes the feedback from its action. In our case, this might be "The average weight of a Border Collie is 30-55 pounds."

This pattern enables the agent to decompose complex tasks into manageable steps and adapt its strategy based on new information.

Our Agent class will implement this pattern by maintaining a conversation history (`self.messages`) and providing methods to interact with the Claude model (`__call__` and `execute`).


In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.system = [{"text": self.system}]
        self.bedrock_client = boto3.client(service_name="bedrock-runtime")

    def __call__(self, message):
        self.messages.append({"role": "user", "content": [{"text": message}]})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": [{"text": result}]})
        return result

    def execute(self):
        inference_config = {
            "temperature": 0.0,
            "stopSequences": [
                "<PAUSE>"
            ],  # we will explore later why this is important!
        }

        additional_model_fields = {"top_k": 200}

        response = self.bedrock_client.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=self.messages,
            system=self.system,
            inferenceConfig=inference_config,
            additionalModelRequestFields=additional_model_fields,
        )
        return response["output"]["message"]["content"][0]["text"]

## Crafting the Prompt

The prompt serves as a set of instructions for the AI model, crucial in defining its behavior and available actions.

In our implementation, we're directing the model to:

- Adhere to the ReAct pattern (Thought, Action, Observation cycle)
- Utilize specific formats for each step (e.g., prefixing thoughts with "Thought:")
- Restrict itself to the provided actions (in this case, a calculator and a dog weight lookup function)

We also include a sample interaction to demonstrate the expected response format. This is analogous to providing a completed template before asking someone to fill out a complex form.

As you can see from the prompt, the agent class, and the inference parameters, we are asking the model to stop generating after it predicts the `<PAUSE>` token. However, it is always better to be safe than sorry, so we add `<PAUSE>` to the `stopSequences`, ensuring the end of token generation after that!


In [ ]:
prompt = """
You run in a loop of Thought, Action, <PAUSE>, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

If available, always call a tool to inform your decisions, never use your parametric knowledge when a tool can be called. 

When you have decided that you need to call a tool, output <PAUSE> and stop thereafter! 

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
<PAUSE>
----- execution stops here -----
You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

## Implementing Helper Functions

To empower our agent with practical capabilities, we'll define several helper functions. These functions will serve as the "actions" our agent can perform. In this example, we're providing:

1. A basic calculator function
2. A function to retrieve average dog weights

In a more sophisticated application, these functions could cover a diverse range of operations, from web scraping to database queries to API calls. They are the agent's versatile interface with external data sources and systems, offering a wide range of possibilities.


In [ ]:
def calculate(what):
    return eval(what, {"__builtins__": {}}, {})


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

## Testing the Agent

With our agent and its action set defined, we'll conduct an initial test using a straightforward query about a toy poodle's weight.

This test will illuminate the agent's information processing flow:

1. It will reason about the necessary steps (identifying the need to look up the weight)
2. It will execute an action (invoking the `average_dog_weight` function)
3. It will process the observation (the returned weight of a toy poodle)
4. It will synthesize this information into a coherent response


In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

In [ ]:
result = average_dog_weight("Toy Poodle")
result

In [ ]:
next_prompt = "Observation: {}".format(result)

In [ ]:
abot(next_prompt)

In [ ]:
abot.messages

In [ ]:
abot = Agent(prompt)
abot.messages

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(eval("37 + 20"), {"__builtins__": {}}, {})
print(next_prompt)

In [ ]:
abot(next_prompt)

### A word on stopSequences

Try to remove the `stopSequence` parameter from the above agent class.

Now think about a few of the questions below before continuing:

- How does your agent perform now?
- When should you use `stopSequences`, when could they become a burden for your application?
- Does the prompt comply with the [prompting standard of Anthropic](https://docs.anthropic.com/en/docs/prompt-engineering)?

At the end of the notebook, try to complete the exercise so that the agent follows your instructions without having to use `stopSequences`.


## Implementing the Reasoning Loop

To enhance our agent's autonomy, we'll implement an iterative loop that enables it to reason, act, and observe multiple times in pursuit of an answer. This loop will continue until the agent reaches a conclusion or hits a predefined maximum number of iterations.

This approach mirrors how a human expert might tackle a complex problem, gathering information and working through multiple steps until arriving at a solution. The loop empowers our agent to handle more intricate queries that demand multiple steps or data points.


In [ ]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

## Final Evaluation

To conclude, we'll test our fully-implemented agent with a more complex query requiring multiple steps of reasoning and action. We'll task it with calculating the combined weight of two distinct dog breeds.

This comprehensive test will showcase the agent's ability to:

1. Deconstruct a complex query into manageable sub-tasks
2. Retrieve information for multiple breeds
3. Perform calculations using the gathered data
4. Integrate all of this information into a coherent final response

By working through this practical example, you'll gain valuable insights into the construction of AI agents capable of solving multi-step problems. Moreover, you'll see firsthand how Amazon Bedrock and model providers like Anthropic's Claude can be effectively utilized. This knowledge will empower you to develop more flexible and diverse AI applications in your future projects.


In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

# Exercise - Rewrite the Agent to use Anthropic style prompting!
